In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
data = pd.read_excel("2008 a 2016_PREFEITURA.xlsx", sheetname="2010")

In [34]:
data.drop(['NOME','NASCIMENTO'], axis=1).head()

,ANO,MÊS,REGIÃO,MATRÍCULA,IDADE AGORA,IDADE NA ÉPOCA,DATA ATUAL,SEXO,COL,HDL,LDL,TRIG
0,2010,1,58.0,NaN,58.0,51.0,2017-09-11 08:08:06.020,F,200,45,126.0,193
1,2010,1,58.0,NaN,25.0,18.0,2017-09-11 08:08:06.020,M,156,57,80.0,135
2,2010,1,60.0,NaN,60.0,53.0,2017-09-11 08:08:06.020,M,165,48,103.0,NaN
3,2010,1,38.0,NaN,55.0,48.0,2017-09-11 08:08:06.020,M,238,49,143.0,318
4,2010,1,5.0,NaN,85.0,78.0,2017-09-11 08:08:06.020,F,198,42,127.0,125


In [35]:
data.shape

(161338, 14)

# Empilhando em uma base só

Um critério para que funcione é criar o vetor, na mão, de "sheetnames", no nosso caso são os anos, de 2008 à 2016

In [36]:
sheets = [str(2008+w) for w in range(0,9)]
sheets

['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [37]:
data = []
for sheet in sheets:
    data.append(pd.read_excel("2008 a 2016_PREFEITURA.xlsx", sheetname=sheet))

Para checar se as bases foram lidas corretamente vamos printar algumas informações

In [38]:
for i in range(0,9):
    unique = data[i].ANO.unique()
    year = data[i].ANO.values[0]
    shape = data[i].shape
    
    print("\t A base ", i," tem ", unique, " valores unicos de ANO. Sendo o primeiro ano o de: ", year, " . A base tem tamanho: ", shape,". \n")

	 A base  0  tem  [2008]  valores unicos de ANO. Sendo o primeiro ano o de:  2008  . A base tem tamanho:  (123671, 14) . 

	 A base  1  tem  [2009]  valores unicos de ANO. Sendo o primeiro ano o de:  2009  . A base tem tamanho:  (147372, 14) . 

	 A base  2  tem  [2010]  valores unicos de ANO. Sendo o primeiro ano o de:  2010  . A base tem tamanho:  (161338, 14) . 

	 A base  3  tem  [2011]  valores unicos de ANO. Sendo o primeiro ano o de:  2011  . A base tem tamanho:  (164472, 14) . 

	 A base  4  tem  [2012]  valores unicos de ANO. Sendo o primeiro ano o de:  2012  . A base tem tamanho:  (148739, 14) . 

	 A base  5  tem  [2013]  valores unicos de ANO. Sendo o primeiro ano o de:  2013  . A base tem tamanho:  (145930, 14) . 

	 A base  6  tem  [2014]  valores unicos de ANO. Sendo o primeiro ano o de:  2014  . A base tem tamanho:  (173730, 14) . 

	 A base  7  tem  [2015]  valores unicos de ANO. Sendo o primeiro ano o de:  2015  . A base tem tamanho:  (161662, 14) . 

	 A base  8  tem

Salvando em CSV cas bases por ano

In [39]:
for i in range(0,9):
    file = "PREFEITURA" + str(sheets[i]) + ".csv"
    data[i].to_csv(file, index = False)

In [40]:
# # Checando se deu certo, o encoding muda pois tem acento e esses caracteres diferentes
# teste = pd.read_csv("PREFEITURA2008.csv", encoding='latin-1')
# teste.shape

Criando uma base consolidada com todos os anos, talvez seja melhor para se trabalhar com a base, talvez fique pesado demais

In [41]:
full = data[0]

for i in range(1,9):
    full = pd.concat([full, data[i]], axis = 0)

In [42]:
full.to_csv("PREFEITURA_FULL.csv", index = False)

# Aqui começa as análises

Primeiro vamos tratar as colunas, retirar acentos das variáveis, e tratar variáveis de datas

Tratando o nome das variáveis

In [43]:
full.rename(columns={'MÊS': 'MES', 'REGIÃO': 'REGIAO', 'MATRÍCULA': 'MATRICULA', 'IDADE NA ÉPOCA':'IDADE_NA_EPOCA', 'IDADE AGORA': 'IDADE_AGORA', 'DATA ATUAL':'DATA_ATUAL'}, inplace=True)

In [44]:
full.drop(['NOME','NASCIMENTO'], axis=1).head(1)

,ANO,MES,REGIAO,MATRICULA,IDADE_AGORA,IDADE_NA_EPOCA,DATA_ATUAL,SEXO,COL,HDL,LDL,TRIG
0,2008,2,1,1768417,79.0,70.0,2017-09-11 08:08:06.460,F,333,65,243,203


Checando e tratando missings


In [45]:
full.isnull().sum()

ANO                   0
MES                   0
REGIAO               13
MATRICULA         62800
NOME                  0
NASCIMENTO         5302
IDADE_AGORA        5856
IDADE_NA_EPOCA     5872
DATA_ATUAL         3462
SEXO                  0
COL               12157
HDL               51796
LDL               66834
TRIG              52195
dtype: int64

In [46]:
types = full.dtypes.values
vars = full.columns.values

In [47]:
types

array([dtype('int64'), dtype('int64'), dtype('O'), dtype('O'), dtype('O'),
       dtype('O'), dtype('float64'), dtype('float64'), dtype('<M8[ns]'),
       dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')], dtype=object)

In [48]:
# full = full.fillna(-1000)

In [49]:
full.isnull().sum()

ANO                   0
MES                   0
REGIAO               13
MATRICULA         62800
NOME                  0
NASCIMENTO         5302
IDADE_AGORA        5856
IDADE_NA_EPOCA     5872
DATA_ATUAL         3462
SEXO                  0
COL               12157
HDL               51796
LDL               66834
TRIG              52195
dtype: int64

Tratando variáveis de data

In [50]:
import time
import datetime

In [51]:
date_nasc = pd.DatetimeIndex(full['NASCIMENTO'])
date_atual = pd.DatetimeIndex(full['DATA_ATUAL'])

In [52]:
full['NASC_DATA'] = date_nasc.strftime("%Y%m%d")
full['ATUAL_DATA'] = date_atual.strftime("%Y%m%d")

In [53]:
full.drop(['NOME','NASCIMENTO'], axis=1).head()

,ANO,MES,REGIAO,MATRICULA,IDADE_AGORA,IDADE_NA_EPOCA,DATA_ATUAL,SEXO,COL,HDL,LDL,TRIG,NASC_DATA,ATUAL_DATA
0,2008,2,1,1768417,79.0,70.0,2017-09-11 08:08:06.460,F,333,65,243,203,19380602,20170911
1,2008,2,19,1741799,74.0,65.0,2017-09-11 08:08:06.460,M,208,39,143,164,19430505,20170911
2,2008,2,23,1777785,85.0,76.0,2017-09-11 08:08:06.460,M,157,33,104,85,19320615,20170911
3,2008,2,2,1914561,52.0,43.0,2017-09-11 08:08:06.460,F,228,48,163,112,19650827,20170911
4,2008,2,27,1417665,56.0,47.0,2017-09-11 08:08:06.460,F,224,36,159,NaN,19610216,20170911


Colocando a data na forma AnoMesDia, a informação vira um número crescente conforme a data é mais atual. É uma forma preliminar de se lidar com variável de data

Tratar datas do tipo 'NaT'

In [54]:
full = full.fillna(-1000)
full['NASC_DATA'] = full['NASC_DATA'].replace(['NaT'], '-1000')
full['ATUAL_DATA'] = full['ATUAL_DATA'].replace(['NaT'], '-1000')

In [55]:
full['NASC_DATA'] = full['NASC_DATA'].astype(int)
full['ATUAL_DATA'] = full['ATUAL_DATA'].astype(int)

In [56]:
full.drop(['NOME','NASCIMENTO'], axis=1).head(2)

,ANO,MES,REGIAO,MATRICULA,IDADE_AGORA,IDADE_NA_EPOCA,DATA_ATUAL,SEXO,COL,HDL,LDL,TRIG,NASC_DATA,ATUAL_DATA
0,2008,2,1,1768417,79.0,70.0,2017-09-11 08:08:06.460,F,333,65,243,203,19380602,20170911
1,2008,2,19,1741799,74.0,65.0,2017-09-11 08:08:06.460,M,208,39,143,164,19430505,20170911


In [57]:
full.isnull().sum()

ANO               0
MES               0
REGIAO            0
MATRICULA         0
NOME              0
NASCIMENTO        0
IDADE_AGORA       0
IDADE_NA_EPOCA    0
DATA_ATUAL        0
SEXO              0
COL               0
HDL               0
LDL               0
TRIG              0
NASC_DATA         0
ATUAL_DATA        0
dtype: int64

Coletando algumas métricas das variáveis e suas descrições, lembrando que valores = -1000 são missings

In [58]:
# full.to_csv("PREFEITURA_FULL.csv", index = False) Atualiza a base consolidada com os tratamentos até essa celula

In [59]:
(full == -1000).sum()

ANO                   0
MES                   0
REGIAO               13
MATRICULA         62800
NOME                  0
NASCIMENTO         5302
IDADE_AGORA        5856
IDADE_NA_EPOCA     5872
DATA_ATUAL         3462
SEXO                  0
COL               12157
HDL               51796
LDL               66834
TRIG              52195
NASC_DATA          5302
ATUAL_DATA         3462
dtype: int64

In [60]:
for var in full.columns:
    str_tipo = str(full[var].dtype)
    print('\t A variável %s é do tipo %s\n' % (var,str_tipo) )

	 A variável ANO é do tipo int64

	 A variável MES é do tipo int64

	 A variável REGIAO é do tipo object

	 A variável MATRICULA é do tipo object

	 A variável NOME é do tipo object

	 A variável NASCIMENTO é do tipo object

	 A variável IDADE_AGORA é do tipo float64

	 A variável IDADE_NA_EPOCA é do tipo float64

	 A variável DATA_ATUAL é do tipo datetime64[ns]

	 A variável SEXO é do tipo object

	 A variável COL é do tipo object

	 A variável HDL é do tipo object

	 A variável LDL é do tipo object

	 A variável TRIG é do tipo object

	 A variável NASC_DATA é do tipo int32

	 A variável ATUAL_DATA é do tipo int32



Análises para metadados